In [2]:
import os
import re
import numpy as np
from tqdm import tqdm

import tensorflow as tf
from transformers import TFElectraModel, ElectraTokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import pandas as pd
import matplotlib.pyplot as plt

c:\project_huiyeol\project\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tf.random.set_seed(42)
np.random.seed(42)

In [4]:
CLASS_NUMBER = 6
BATCH_SIZE = 10
NUM_EPOCHS = 100
VALID_SPLIT = 0.2
BERT_CKPT = './data_out/'
DATA_IN_PATH = '../metadata/'
DATA_OUT_PATH = './data_out/'

In [5]:
class TFElectraClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super().__init__()

        self.bert = TFElectraModel.from_pretrained(model_name, cache_dir=dir_path, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.layers.Dense(num_class, name='classifier', activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range))

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = outputs[0]
        last_hidden_state = self.flatten(last_hidden_state)
        last_hidden_state = self.dropout(last_hidden_state, training=training)
        logits = self.classifier(last_hidden_state)

        return logits

In [6]:
cls_model = TFElectraClassifier(model_name='monologg/koelectra-base-v3-discriminator', dir_path=os.path.join(BERT_CKPT, 'model'), num_class=CLASS_NUMBER)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'electra.embeddings.position_ids', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing TFElectraModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further train

In [7]:
lyrics1 = pd.read_csv(DATA_IN_PATH+'Song_Lyrics.csv')
lyrics2 = pd.read_csv(DATA_IN_PATH+'Song_Lyrics2.csv')

In [8]:
tag_df = pd.read_csv(DATA_IN_PATH+'newtag_1118_.csv', encoding='cp949')

In [9]:
tag_df.rename(columns = {'song_id':'SONG_ID'}, inplace=True)

In [10]:
total = pd.merge(lyrics2, tag_df, left_on='SONG_ID', right_on='SONG_ID')

In [11]:
total.head()

,IDX_x,SONG_ID,LYRICS,Unnamed: 0,IDX_y,song_title,대분류,중분류
0,14,2466,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...,2061,10669994.0,오늘도 난,신나는,신나는
1,163,5112,제각기 있기 였기 때문이기 \n사랑했기 때문이기 \n하루지기 이야기 이야기 \n잊지...,2062,10225315.0,끝 (End),신나는,흥겨운
2,316,7712,당신은 모르실 거야\n얼마나 사랑했는지\n세월이 흘러가면은\n그때서 뉘우칠 거야\n...,2063,11485381.0,당신은 모르실거야 (CF - LIG 핑클편),차분한,잔잔한
3,464,9270,이렇게 멋진 파란 하늘 위로 \n날으는 마법 융단을 타고 \n이렇게 멋진 푸른 세상...,2064,8327385.0,매직 카펫 라이드,밝은,발랄한
4,486,9302,\N,2065,11503684.0,Deep In The Night,차분한,평온한


In [56]:
tag_df

,Unnamed: 0,IDX,SONG_ID,song_title,대분류,중분류
0,0,57088.0,4203068.0,소녀,차분한,평온한
1,1,258244.0,4203787.0,다알아,밝은,희망찬
2,2,57038.0,4209541.0,기억속의 먼 그대에게,우울한,슬픈
3,3,156375.0,4214654.0,이별하면,우울한,외로운
4,4,56764.0,4230004.0,I'm Fine Thank You,우울한,슬픈
...,...,...,...,...,...,...
3288,3288,20349965.0,34850634.0,사랑인가 봐,밝은,발랄한 희망찬
3289,3289,20350574.0,34890162.0,다시 만날까 봐,우울한,슬픈 외로운
3290,3290,20351090.0,34927767.0,정이라고 하자 (Feat. 10CM),차분한,잔잔한 평온한
3291,3291,20351312.0,34943312.0,나의 X에게,밝은,발랄한


In [12]:
data = total[['SONG_ID','song_title' ,'LYRICS', '대분류']]

In [13]:
data

,SONG_ID,song_title,LYRICS,대분류
0,2466,오늘도 난,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...,신나는
1,5112,끝 (End),제각기 있기 였기 때문이기 \n사랑했기 때문이기 \n하루지기 이야기 이야기 \n잊지...,신나는
2,7712,당신은 모르실거야 (CF - LIG 핑클편),당신은 모르실 거야\n얼마나 사랑했는지\n세월이 흘러가면은\n그때서 뉘우칠 거야\n...,차분한
3,9270,매직 카펫 라이드,이렇게 멋진 파란 하늘 위로 \n날으는 마법 융단을 타고 \n이렇게 멋진 푸른 세상...,밝은
4,9302,Deep In The Night,\N,차분한
...,...,...,...,...
3249,35353251,열이올라요 (Heart Burn),그대 아무런 말도 하지 마요\n이 맘은 여전히 그대로예요\n따가운 햇살 그 아래 우...,차분한
3250,35353557,없었던 일로,대충 모자를 눌러 쓰고서\n부은 눈을 가리고 나와\n오늘부터는 울어도\n나 때문에 ...,차분한
3251,35380072,물개,지금 시각 열한 시 이십팔 분\n물론 아니지 아침\n하루 일과 다 마치구\n우연히 ...,차분한
3252,35389041,Sparkling,"Come and, talk to me baby\n뭐든 콕 찝어 말을 해줘\n사소한 ...",신나는


In [15]:
data.isnull().sum()

SONG_ID       0
song_title    0
LYRICS        4
대분류           0
dtype: int64

In [16]:
data = data.dropna()

In [17]:
def preprocessing(x):
    temp = x.split('\n')
    temp = ' '.join(temp)
    return temp

In [18]:
data['Edited_LYRICS'] = data['LYRICS'].apply(preprocessing)

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
def slicing(x):
    x = x.split()
    res = []
    for i in range(0, len(x)+1, 120):
        if len(x)-i < 120:
            temp = x[i:]
            res.append(temp)
            break
        temp = x[i:i+120]
        res.append(temp)
    return res

In [20]:
data['slice'] = data['Edited_LYRICS'].apply(slicing)

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
length = data['Edited_LYRICS'].apply(lambda x:len(x))
length.max()

8696

In [22]:
data['Sliced_LYRICS'] = ''

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
new_data = pd.DataFrame(data.iloc[0][:]).T
for idx in range(len(data)):
    try:
        for i in range(len(data.loc[idx]['slice'])):
            temp = data.loc[idx]['slice'][i]
            txt = ' '.join(temp)
            data.loc[idx, 'Sliced_LYRICS'] = txt
            new_data = pd.concat([new_data, pd.DataFrame(data.loc[idx]).T])
    except:
        continue

c:\project_huiyeol\project\venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [47]:
new_data[new_data['song_title']=='진심']

,SONG_ID,song_title,LYRICS,대분류,Edited_LYRICS,slice,Sliced_LYRICS


In [24]:
new_data

,SONG_ID,song_title,LYRICS,대분류,Edited_LYRICS,slice,Sliced_LYRICS
0,2466,오늘도 난,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...,신나는,나도 알지 못한날 굳이 알려고 들지마 보여줄 수 없는 내 마음만 안타까울 뿐 이...,"[[나도, 알지, 못한날, 굳이, 알려고, 들지마, 보여줄, 수, 없는, 내, 마음...",
0,2466,오늘도 난,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...,신나는,나도 알지 못한날 굳이 알려고 들지마 보여줄 수 없는 내 마음만 안타까울 뿐 이...,"[[나도, 알지, 못한날, 굳이, 알려고, 들지마, 보여줄, 수, 없는, 내, 마음...",나도 알지 못한날 굳이 알려고 들지마 보여줄 수 없는 내 마음만 안타까울 뿐 이런 ...
1,5112,끝 (End),제각기 있기 였기 때문이기 \n사랑했기 때문이기 \n하루지기 이야기 이야기 \n잊지...,신나는,제각기 있기 였기 때문이기 사랑했기 때문이기 하루지기 이야기 이야기 잊지않기 ...,"[[제각기, 있기, 였기, 때문이기, 사랑했기, 때문이기, 하루지기, 이야기, 이야...",제각기 있기 였기 때문이기 사랑했기 때문이기 하루지기 이야기 이야기 잊지않기 제각기...
1,5112,끝 (End),제각기 있기 였기 때문이기 \n사랑했기 때문이기 \n하루지기 이야기 이야기 \n잊지...,신나는,제각기 있기 였기 때문이기 사랑했기 때문이기 하루지기 이야기 이야기 잊지않기 ...,"[[제각기, 있기, 였기, 때문이기, 사랑했기, 때문이기, 하루지기, 이야기, 이야...",이야기 이야기 끝이여 끝이 아니길 빌어 끝에 저 끝에 내가 보내야 하나 기다릴테니 ...
2,7712,당신은 모르실거야 (CF - LIG 핑클편),당신은 모르실 거야\n얼마나 사랑했는지\n세월이 흘러가면은\n그때서 뉘우칠 거야\n...,차분한,당신은 모르실 거야 얼마나 사랑했는지 세월이 흘러가면은 그때서 뉘우칠 거야 When...,"[[당신은, 모르실, 거야, 얼마나, 사랑했는지, 세월이, 흘러가면은, 그때서, 뉘...",당신은 모르실 거야 얼마나 사랑했는지 세월이 흘러가면은 그때서 뉘우칠 거야 When...
...,...,...,...,...,...,...,...
3248,35333345,POP!,What's wrong?\n홀릴 듯이 난 너를 자극해 (Pop pop pop) \n...,신나는,What's wrong? 홀릴 듯이 난 너를 자극해 (Pop pop pop) Wa...,"[[What's, wrong?, 홀릴, 듯이, 난, 너를, 자극해, (Pop, po...",What's wrong? 홀릴 듯이 난 너를 자극해 (Pop pop pop) Wat...
3248,35333345,POP!,What's wrong?\n홀릴 듯이 난 너를 자극해 (Pop pop pop) \n...,신나는,What's wrong? 홀릴 듯이 난 너를 자극해 (Pop pop pop) Wa...,"[[What's, wrong?, 홀릴, 듯이, 난, 너를, 자극해, (Pop, po...",보여 아무리 숨겨도 이미 들킨 걸 자꾸 둥둥 떠다니잖아 여유롭게 Check it 보...
3248,35333345,POP!,What's wrong?\n홀릴 듯이 난 너를 자극해 (Pop pop pop) \n...,신나는,What's wrong? 홀릴 듯이 난 너를 자극해 (Pop pop pop) Wa...,"[[What's, wrong?, 홀릴, 듯이, 난, 너를, 자극해, (Pop, po...",됐어 Four! 딱 숨을 멈춰 Three! 난 너를 겨눠 Two! One! Here...
3249,35353251,열이올라요 (Heart Burn),그대 아무런 말도 하지 마요\n이 맘은 여전히 그대로예요\n따가운 햇살 그 아래 우...,차분한,그대 아무런 말도 하지 마요 이 맘은 여전히 그대로예요 따가운 햇살 그 아래 우리 ...,"[[그대, 아무런, 말도, 하지, 마요, 이, 맘은, 여전히, 그대로예요, 따가운,...",그대 아무런 말도 하지 마요 이 맘은 여전히 그대로예요 따가운 햇살 그 아래 우리 ...


In [25]:
temp = []
for sliced in new_data.loc[0, 'slice']:
    try:
        for i in sliced:
            txt = ' '.join(i)
            temp.append(txt)
    except:
        continue


In [26]:
length.mean()

821.8910769230769

In [27]:
MAX_LEN = 120

In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['대분류'] = le.fit_transform(data['대분류'])

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
import pickle
with open('encoder.pickle', 'wb') as f:
    pickle.dump(le, f)

In [30]:
from sklearn.model_selection import train_test_split

data_train, data_valid = train_test_split(data, test_size=VALID_SPLIT, stratify=data['대분류'], random_state = 42)

x_train = data_train['Sliced_LYRICS']
y_train = data_train['대분류']
x_valid = data_valid['Sliced_LYRICS']
y_valid = data_valid['대분류']

In [31]:
import pickle

with open(DATA_OUT_PATH+'bert_tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

In [32]:
def bert_tokenizer(sent, MAX_LEN):
    encoded_dict = loaded_tokenizer.encode_plus(
        text=sent,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True
    )

    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']

    return input_id, attention_mask, token_type_id

In [33]:
input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []

for train_sent, train_label in tqdm(zip(x_train, y_train), total=len(x_train)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAX_LEN)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

100%|██████████| 2600/2600 [00:02<00:00, 1258.92it/s]


In [34]:
train_input_ids = np.array(input_ids, dtype=int)
train_attention_masks = np.array(attention_masks, dtype=int)
train_type_ids = np.array(token_type_ids, dtype=int)
train_inputs = (train_input_ids, train_attention_masks, train_type_ids)

train_data_labels = np.asarray(train_data_labels, dtype=np.int32)

print('# sents: {}, # labels" {}'.format(len(train_input_ids), len(train_data_labels)))

# sents: 2600, # labels" 2600


In [35]:
optimizer = tf.keras.optimizers.Adam(3e-6)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [66]:
data_valid[data_valid['SONG_ID']==32473998]

,SONG_ID,song_title,LYRICS,대분류,Edited_LYRICS,slice,Sliced_LYRICS
2594,32473998,좋은 사람 있으면 소개시켜줘,좋은 사람 있으면 소개시켜줘\n때로는 물처럼 때로는 불처럼\n진심으로 나만을 사랑할...,2,좋은 사람 있으면 소개시켜줘 때로는 물처럼 때로는 불처럼 진심으로 나만을 사랑할 수...,"[[좋은, 사람, 있으면, 소개시켜줘, 때로는, 물처럼, 때로는, 불처럼, 진심으로...",저런 때 있었다며 새로운 사람이 그리운걸까 옛사람이 그리워진걸까 옛사람이 그리워진걸까


In [33]:
!nvidia-smi

Thu Nov 24 11:07:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   50C    P8     5W /  N/A |   3292MiB /  4096MiB |     31%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [67]:
a, b, c = bert_tokenizer('안녕하세요', MAX_LEN)

In [68]:
cls_model.call((np.array(a).reshape(1,-1), np.array(b).reshape(1,-1), np.array(c).reshape(1,-1)))

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[6.0232687e-03, 9.9397594e-01, 1.2761201e-07, 3.7010686e-13,
        1.7939812e-10, 7.6587662e-07]], dtype=float32)>

In [69]:
cls_model.built = True

In [70]:
cls_model.load_weights(DATA_OUT_PATH+'weights.h5')

In [71]:
cls_model.summary()

Model: "tf_electra_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_electra_model (TFElectra  multiple                 112330752 
 Model)                                                          
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 classifier (Dense)          multiple                  552966    
                                                                 
Total params: 112,883,718
Trainable params: 112,883,718
Non-trainable params: 0
_________________________________________________________________


In [72]:
input_ids = []
attention_masks = []
token_type_ids = []
test_data_labels = []

for test_sent, test_label in tqdm(zip(x_valid[:], y_valid[:])):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAX_LEN)
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        test_data_labels.append(test_label)
    except:
        pass

test_input_ids = np.array(input_ids, dtype=int)
test_attention_masks = np.array(attention_masks, dtype=int)
test_type_ids = np.array(token_type_ids, dtype=int)
test_inputs = (test_input_ids, test_attention_masks, test_type_ids)

test_data_labels = np.asarray(test_data_labels, dtype=np.int32)

650it [00:00, 1110.29it/s]


In [73]:
prediction = cls_model.predict(test_inputs)

21/21 [==============================] - 15s 442ms/step


In [94]:
prediction.shape

(650, 6)

In [83]:
le.inverse_transform([0,1,2,3,4,5])

array(['공격적인', '긴장감', '밝은', '신나는', '우울한', '차분한'], dtype=object)

In [81]:
prediction[0]

array([1.10259534e-04, 6.29549686e-05, 3.55637079e-04, 1.96719353e-04,
       8.32887650e-01, 1.66386694e-01], dtype=float32)

In [87]:
sorted(prediction[0])

[6.295497e-05,
 0.000110259534,
 0.00019671935,
 0.00035563708,
 0.1663867,
 0.83288765]

In [77]:
np.argmax(prediction[0])

4

In [78]:
test_data_labels[0]

2

In [86]:
le.inverse_transform(np.argsort(prediction[0]))

array(['긴장감', '공격적인', '신나는', '밝은', '차분한', '우울한'], dtype=object)

In [89]:
res = 0
fail = 0

for pred, label in zip(prediction, test_data_labels):
    if label in np.argsort(pred)[-2:] and (pred[np.argsort(pred)[-2]] >= 0.4 or pred[np.argsort(pred)[-1]] >= 0.4):
        res += 1
    else:
        pass
    fail += 1

print(res/fail*100)

66.66666666666666


In [173]:
lyrics = preprocessing(data_train.loc[5, 'LYRICS'])
print(lyrics)

It's on and bump it  It's on and bump it  It's on and bump it  get ready to ride yeah It's on and bump it baby come on and bump it baby It's on and bump it baby  come on and bump it baby people in the house lets go  아련히 기억에 멀어지던  너를 다시 볼수 있어  행복했었어  하지만 나 멀리서 너만을  하염없이 보고 말았지  우리 헤어지고 나의 일기 속에  너의 이름으로 채운적도 많았고  매일 밤 나의 옷깃을  적시며 너의 이름  부른 적 많았어  Get ready it`s on  영원히 나와 같이 있을거라던  그런날들은 이제 모두  Done Done  그런 넌 넌 내겐 모든 것이였던  떠나버린 넌 너무나 멀어진  넌 왜 날 떠나갔나  오직 너를 위해 살아온 나  왜 나를 잊으려고 하나  너와나 uh why you gotta leave  me now  가슴에 너를 묻어 버리고  기억에서 너를 지웠었는데  다시 흔들리는 나를 어떻게  해야 해  우리 다시는 헤어지지 말자고  우리 사랑 포기하지 말자고  말하고 싶어 너를 내게  돌리고 싶어 나만을 안아줘  이대로 나를 잊어줘 문득  떠오르는 나의 모습 지워줘  힘들어도 다신 네곁에 갈순  없잖아 날 용서해  그저 행복했던 날들 너를  위해 살아온  나는 절대 너를 원망하지 않아  이젠 다시 나를 받아줘 내게  돌아올수 없니  uh tell me the reason why  you left me  baby take me back  어서 나를 받아줘 언제나 너를 기다릴께 가슴에 너를 묻어 버리고  기억에서 너를 지웠었는데 다시 흔들리는 나를 어떻게  해야 해 우리 다시는 헤어지지 말자고 우리 사랑 포기하지 말자고 말하고 싶어 너를 내게 돌리고  싶어 나만을 안아줘 우리 다시는 헤어지지 말자고 우리 사랑 포기하지 말자고 말하고 싶어 너를 내게 돌리고 

In [174]:
def slicing(x):
    x = x.split()
    print(x)
    res = []
    for i in range(0, len(x)+1, MAX_LEN):
        if len(x)-i < MAX_LEN:
            temp = x[i:]
            res.append(temp)
            break
        temp = x[i:i+MAX_LEN]
        res.append(temp)
    return res

In [175]:
lyrics = slicing(lyrics)
print(lyrics)

["It's", 'on', 'and', 'bump', 'it', "It's", 'on', 'and', 'bump', 'it', "It's", 'on', 'and', 'bump', 'it', 'get', 'ready', 'to', 'ride', 'yeah', "It's", 'on', 'and', 'bump', 'it', 'baby', 'come', 'on', 'and', 'bump', 'it', 'baby', "It's", 'on', 'and', 'bump', 'it', 'baby', 'come', 'on', 'and', 'bump', 'it', 'baby', 'people', 'in', 'the', 'house', 'lets', 'go', '아련히', '기억에', '멀어지던', '너를', '다시', '볼수', '있어', '행복했었어', '하지만', '나', '멀리서', '너만을', '하염없이', '보고', '말았지', '우리', '헤어지고', '나의', '일기', '속에', '너의', '이름으로', '채운적도', '많았고', '매일', '밤', '나의', '옷깃을', '적시며', '너의', '이름', '부른', '적', '많았어', 'Get', 'ready', 'it`s', 'on', '영원히', '나와', '같이', '있을거라던', '그런날들은', '이제', '모두', 'Done', 'Done', '그런', '넌', '넌', '내겐', '모든', '것이였던', '떠나버린', '넌', '너무나', '멀어진', '넌', '왜', '날', '떠나갔나', '오직', '너를', '위해', '살아온', '나', '왜', '나를', '잊으려고', '하나', '너와나', 'uh', 'why', 'you', 'gotta', 'leave', 'me', 'now', '가슴에', '너를', '묻어', '버리고', '기억에서', '너를', '지웠었는데', '다시', '흔들리는', '나를', '어떻게', '해야', '해', '우리', '다시는', '헤어지지', '말자고', '우리',

In [180]:
lyrics[0]

["It's",
 'on',
 'and',
 'bump',
 'it',
 "It's",
 'on',
 'and',
 'bump',
 'it',
 "It's",
 'on',
 'and',
 'bump',
 'it',
 'get',
 'ready',
 'to',
 'ride',
 'yeah',
 "It's",
 'on',
 'and',
 'bump',
 'it',
 'baby',
 'come',
 'on',
 'and',
 'bump',
 'it',
 'baby',
 "It's",
 'on',
 'and',
 'bump',
 'it',
 'baby',
 'come',
 'on',
 'and',
 'bump',
 'it',
 'baby',
 'people',
 'in',
 'the',
 'house',
 'lets',
 'go',
 '아련히',
 '기억에',
 '멀어지던',
 '너를',
 '다시',
 '볼수',
 '있어',
 '행복했었어',
 '하지만',
 '나',
 '멀리서',
 '너만을',
 '하염없이',
 '보고',
 '말았지',
 '우리',
 '헤어지고',
 '나의',
 '일기',
 '속에',
 '너의',
 '이름으로',
 '채운적도',
 '많았고',
 '매일',
 '밤',
 '나의',
 '옷깃을',
 '적시며',
 '너의',
 '이름',
 '부른',
 '적',
 '많았어',
 'Get',
 'ready',
 'it`s',
 'on',
 '영원히',
 '나와',
 '같이',
 '있을거라던',
 '그런날들은',
 '이제',
 '모두',
 'Done',
 'Done',
 '그런',
 '넌',
 '넌',
 '내겐',
 '모든',
 '것이였던',
 '떠나버린',
 '넌',
 '너무나',
 '멀어진',
 '넌',
 '왜',
 '날',
 '떠나갔나',
 '오직',
 '너를',
 '위해',
 '살아온',
 '나',
 '왜',
 '나를',
 '잊으려고',
 '하나']

In [181]:
temp = []
for lyric in lyrics:
    try:
        txt = ' '.join(lyric)
        temp.append(txt)
    except:
        continue

In [183]:


input_ids = []
attention_masks = []
token_type_ids = []

for lyric in temp:
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(lyric, MAX_LEN)
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)

    except Exception as e:
        print(e)
        pass
    
test_input_ids = np.array(input_ids, dtype=int)
test_attention_masks = np.array(attention_masks, dtype=int)
test_type_ids = np.array(token_type_ids, dtype=int)
test_inputs = (test_input_ids, test_attention_masks, test_type_ids)

In [185]:
test_inputs[1].shape

(3, 120)

In [186]:
np.array(temp).shape

(3,)

In [187]:
np.array(lyrics).shape

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


(3,)

In [188]:
np.array(lyrics)

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


array([list(["It's", 'on', 'and', 'bump', 'it', "It's", 'on', 'and', 'bump', 'it', "It's", 'on', 'and', 'bump', 'it', 'get', 'ready', 'to', 'ride', 'yeah', "It's", 'on', 'and', 'bump', 'it', 'baby', 'come', 'on', 'and', 'bump', 'it', 'baby', "It's", 'on', 'and', 'bump', 'it', 'baby', 'come', 'on', 'and', 'bump', 'it', 'baby', 'people', 'in', 'the', 'house', 'lets', 'go', '아련히', '기억에', '멀어지던', '너를', '다시', '볼수', '있어', '행복했었어', '하지만', '나', '멀리서', '너만을', '하염없이', '보고', '말았지', '우리', '헤어지고', '나의', '일기', '속에', '너의', '이름으로', '채운적도', '많았고', '매일', '밤', '나의', '옷깃을', '적시며', '너의', '이름', '부른', '적', '많았어', 'Get', 'ready', 'it`s', 'on', '영원히', '나와', '같이', '있을거라던', '그런날들은', '이제', '모두', 'Done', 'Done', '그런', '넌', '넌', '내겐', '모든', '것이였던', '떠나버린', '넌', '너무나', '멀어진', '넌', '왜', '날', '떠나갔나', '오직', '너를', '위해', '살아온', '나', '왜', '나를', '잊으려고', '하나']),
       list(['너와나', 'uh', 'why', 'you', 'gotta', 'leave', 'me', 'now', '가슴에', '너를', '묻어', '버리고', '기억에서', '너를', '지웠었는데', '다시', '흔들리는', '나를', '어떻게', '해야', '해', '우리', 

In [206]:
lyrics2[lyrics2['SONG_ID']==32457751]

,IDX,SONG_ID,LYRICS
3076,289342,32457751,찐찐찐찐 찐이야 완전 찐이야 \n진짜가 나타났다 지금 \n찐찐찐찐 찐이야 완전 찐이...
